# Parallel Computing
<style type="text/css">
  .reveal p {
    text-align: left;
  }
  .reveal ul {
    display: block;
  }
  .reveal ol {
    display: block;
  }
</style>
# Shared Or Distributed Memory
## Two Types of Parallelization
* Broadly speaking there are two types of parallelization architectures:

    * Shared Memory: all processors live on  the same machine and have access to the same memory
   
    * Distributed Memory: processors live on different machines and/or have access to their own memory

* How you can parallelize code depends on what type of machine(s) you are dealing with
    
    * Both have their own advantages and disadvantages

## Shared Memory
* Shared Memory **Advantages**:
   
    * Don't need to worry about communication
   
    * Generally only need to put an indicator in front of a for loop or map

* Shared Memory **Disadvantages**:
   
    * Limited in sclare (only so many processors you can put in one machine)
   
    * Can easily run into race conditions

## Distributed Memory
* Distributed Memory **Advantages**:
   
    * Can, in principle, scale arbitrarily large
   
    * Precise control over communication between processes

* Distributed Memory **Disadvantages**:
   
    * Extra effort on the programmers part

# Shared Memory
## Threads
* There are a lot of different types of shared memory parallel code across languages
   
    * Open MP (most common)
    
    * Pthreads
    
    * etc.

* All generally act in the same way

* In julia this is implemented using the Threads built-in library

* How many threads are active depend on how you started the julia application
`julia --threads 8`

* You can check how many are active using `nthreads()`

In [1]:
println("I have $(Threads.nthreads()) active threads")
println("This is my id $(Threads.threadid())")#Main Thread id is 1

I have 1 active threads
This is my id 1


## Parallelizing A For Loop
* Let's start with a simple for loop

In [2]:
a = zeros(10)
for i in 1:10
    a[i] = i^2
end
println(a)

[1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0]


* We can easily make this for loop run in parallel using `Threads.@threads`

In [3]:
a = zeros(10)
Threads.@threads for i in 1:10
    a[i] = i^2
end
println(a)

[1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0]


* It's just that simple

## Digging deeper into @threads
* Let's dig a little deeper into that macro: what's going on under the hood

In [4]:
a = zeros(10)
id = zeros(10)
Threads.@threads for i in 1:10
    a[i] = i^2
    id[i] = Threads.threadid()
end
println(a)
println(id)

[1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


* There are 8 threads

* Each thread is assigned a portion of the "work"

* Note: there are 10 tasks so threads 1 and 2 do twice the work
    
    * Therefore won't get a full 8x speed up
    
    * Best case scenario is a 4x speed up
    
    * In reality may run slower because there's a small overhead to each paralle for loop


## Race conditions
* In our previous example all the threads are modifying the same vector $a$

* This is a great advantage: 

    * Don't need to worry about how to share memomry across processes

* BUT: what happens if two threads try and modify the same element of $a$
    
* Can run into a race condition:

    * Results may depend on which process modified $a$ first (hence race)

    * Will lead to **unstable** and **incorrect** output

## A Simple example
* Let's start with a simple example where we add 1 to a vecto 10,000 times

In [5]:
acc = zeros(3)
for i in 1:10_000
    acc .+= 1
end
println(acc)

[10000.0, 10000.0, 10000.0]


* Produces exactly what we would expect

    * What happens if we parallelize it?

In [6]:
acc = zeros(3)
Threads.@threads for i in 1:10_000
    acc .+= 1
end
println(acc)

[10000.0, 10000.0, 10000.0]


* Huh?  What happened here?

## Digging Into The Race condition

* All that is going on can be understood by seeing what `acc .+= 1` is doing

* Essentially `acc .+= 1`:
    
    1. Takes a copy of an element `acc`
    
    2. Adds 1 to that copy
    
    3. Overwrites that element of `acc` with its new value

* Now suppose that process 1 starts tasks 1,2,3
    
    * While process 1 is on task 2 process 2 starts task 1,2,3

    * The results of process 1 will be overwritten by process 2

    * That sum won't be preformed

* Generally speaking: make sure that all process are working on their own blocks of memory

    * Usually this is a natural outcome of our code

# Shared Memory Examples

## Example: Monte Carlo

* Take a very simple monte carlo example

* Let $x_t$ follow an AR(1)

* Want a sample of the distribution of $x_t$ after $1000$ periods
    * A sample of size $100,000$

* We'll need to draw $100,000$ draws form this AR(1) process
    * Ideal for parallelization

## AR(1) Code

In [7]:
using Parameters
mutable struct AR1
    μ::Float64 #Mean of the AR(1)
    ρ::Float64 #persistence of the AR(1)
    σ::Float64 #standard deviaiton of the AR(1)
end
"""
   simulateAR1(ar,x0,T)

Simulates an AR(1) ar for T periods with initial value x0
"""
function simulateAR1(ar,x0,T)
    @unpack σ,μ,ρ = ar #note order doesn't matter
    x = zeros(T+1)# initialize
    x[1] = x0
    for t in 1:T
        x[t+1] = (1-ρ)*μ + ρ*x[t] + σ*randn()
    end
    return x[2:end]
end;

## How Does It Behave Serially

In [8]:
#get a sample of 100_000 of x after 100 periods
"""
    draw_endofT_sample(ar,N,T)

Draws N values of x after T periods where x follows AR1 with x[1] = 0.
"""
function draw_endofT_sample(ar,N,T)
    sample = zeros(N)
    for i in 1:N
        sample[i] = simulateAR1(ar,0.,T)[end]
    end
    return sample
end

ar = AR1(1.,.99,0.01)
sample = draw_endofT_sample(ar,100_000,1000)
@time draw_endofT_sample(ar,100_000,1000);

  1.147890 seconds (200.00 k allocations: 1.521 GiB, 16.17% gc time)


## Parallelizing is Easy!

In [9]:
"""
    draw_endofT_sample(ar,N,T)

Draws N values of x after T periods where x follows AR1 with x[1] = 0.
"""
function draw_endofT_sample_parallel(ar,N,T)
    sample = zeros(N)
    Threads.@threads for i in 1:N
        sample[i] = simulateAR1(ar,0.,T)[end]
    end
    return sample
end

sample = draw_endofT_sample_parallel(ar,100_000,1000)
@time draw_endofT_sample_parallel(ar,100_000,1000);

  0.999745 seconds (200.01 k allocations: 1.521 GiB, 16.72% gc time)


* Note a lot of the time is spent in garbage collection
    * i.e. allocating and deallocating memory
    * can we speed this up?

## Write More Efficient Serial Code
* Don't need to save entire AR(1) process 
    * Just current value when simulating

In [10]:
"""
   simulateAR1final(ar,x0,T)

Simulates an AR(1) ar for T periods with initial value x0.
Returns the final period value
"""
function simulateAR1final(ar,x,T)
    @unpack σ,μ,ρ = ar #note order doesn't matter
    for t in 1:T
        x = (1-ρ)*μ + ρ*x + σ*randn()
    end
    return x
end

"""
    draw_endofT_sample(ar,N,T)

Draws N values of x after T periods where x follows AR1 with x[1] = 0.
"""
function draw_endofT_sample(ar,N,T)
    sample = zeros(N)
    for i in 1:N
        sample[i] = simulateAR1final(ar,0.,T)
    end
    return sample
end
sample = draw_endofT_sample(ar,100_000,1000)
@time draw_endofT_sample(ar,100_000,1000);

  0.553427 seconds (2 allocations: 781.328 KiB)


## Speedup Carries Over to Parallel Code

In [11]:
"""
    draw_endofT_sample(ar,N,T)

Draws N values of x after T periods where x follows AR1 with x[1] = 0.
"""
function draw_endofT_sample_parallel(ar,N,T)
    sample = zeros(N)
    Threads.@threads for i in 1:N
        sample[i] = simulateAR1final(ar,0.,T)
    end
    return sample
end
sample = draw_endofT_sample_parallel(ar,100_000,1000)
@time draw_endofT_sample_parallel(ar,100_000,1000);

  0.545930 seconds (10 allocations: 781.844 KiB)


## Value Function Iteration
* Another easily parallelizable algorith is value function iteration

* Need to evaluate Bellman Map at all of the interpolation gridpoints
    * Can be executed in parallel

* Let's see this applied to our code soling the RBC model

## RBC Model

In [12]:
using BasisMatrices,Optim,LinearAlgebra
@with_kw mutable struct NCParameters
    A::Float64 = 1.
    α::Float64 = 0.3
    β::Float64 = 0.96
    kgrid::Vector{Float64} = LinRange(0.05,0.5,20)
    spline_order::Int = 3
end;

## Optimal Policy

In [13]:
"""
    optimalpolicy(para::NCParameters,Vprime,k)

Computes optimal policy using continuation value function V and current capital
level k given parameters in para.
"""
function optimalpolicy(para::NCParameters,Vprime,k)
    @unpack A,α,β,kgrid = para
    k_bounds = [kgrid[1],kgrid[end]]
    f_objective(kprime) = -( log(A*k^α-kprime)+β*Vprime(kprime) ) #stores objective as function
    k_max = min(A*k^α-.001,k_bounds[2]) #Can't have negative consumptions
    result = optimize(f_objective,k_bounds[1],k_max)
    return (kprime = result.minimizer,V=-result.minimum) #using named tuples 
end;

## Solving the Bellman Equation

In [14]:
"""
    bellmanmap(Vprime,para::NCParameters)

Apply the bellman map given continuation value function Vprime
"""
function bellmanmap(para::NCParameters,Vprime::Interpoland)
    kbasis = Vprime.basis
    #sometimes it's helpful to tell julia what type a variable is
    knodes = nodes(kbasis)[1]::Vector{Float64}
    V = map(k->optimalpolicy(para,Vprime,k).V,knodes)
    return Interpoland(kbasis,V)
end

"""
    solvebellman(para::NCParameters,V0::Interpoland)

Solves the bellman equation for a given V0
"""
function solvebellman(para::NCParameters,V0::Interpoland)
    diff = 1
    #Iterate of Bellman Map until difference in coefficients goes to zero
    while diff > 1e-6
        V = bellmanmap(para,V0)
        diff = norm(V.coefs-V0.coefs,Inf)
        V0 = V
    end
    kbasis = V0.basis
    knodes = nodes(kbasis)[1]::Vector{Float64}
    #remember optimalpolicy also returns the argmax
    kprime = map(k->optimalpolicy(para,V0,k).kprime,knodes)
    #Now get policies
    return Interpoland(kbasis,kprime),V0
end;

## Serial Benchmark

In [15]:
"""
    getV0(para::NCParameters)

Initializes V0(k) = 0 using the kgrid of para
"""
function getV0(para::NCParameters)
    @unpack kgrid,spline_order = para

    kbasis = Basis(SplineParams(kgrid,0,spline_order))

    return Interpoland(kbasis,k->0 .*k)
end
para = NCParameters()
para.kgrid = LinRange(0.05,0.5,100)
kprime,V = solvebellman(para,getV0(para))
@time solvebellman(para,getV0(para));

  1.638470 seconds (18.75 M allocations: 1.866 GiB, 16.64% gc time)


## How To Parallelize?
*  `V = map(k->optimalpolicy(para,Vprime,k).V,knodes)` evaluates the optimal policy
    * at every point in the state space

* Want: to do this in parallel
    * for a given `Vprime` all the evaluations are independent

* The `map` function is just a for loop so we can use Threads

* The library `ThreadTools` provides us with a convenient way of doing this
    * Replace `map` with `tmap`


## Parallelizing Bellman Map

In [16]:
using ThreadTools
"""
    bellmanmap_parallel(Vprime,para::NCParameters)

Apply the bellman map given continuation value function Vprime
"""
function bellmanmap_parallel(para::NCParameters,Vprime::Interpoland)
    kbasis = Vprime.basis
    #sometimes it's helpful to tell julia what type a variable is
    knodes = nodes(kbasis)[1]::Vector{Float64}
    V = tmap(k->optimalpolicy(para,Vprime,k).V,knodes)
    return Interpoland(kbasis,V)
end

"""
    solvebellman_parallel(para::NCParameters,V0::Interpoland)

Solves the bellman equation for a given V0
"""
function solvebellman_parallel(para::NCParameters,V0::Interpoland)
    diff = 1
    #Iterate of Bellman Map until difference in coefficients goes to zero
    while diff > 1e-6
        V = bellmanmap_parallel(para,V0)
        diff = norm(V.coefs-V0.coefs,Inf)
        V0 = V
    end
    kbasis = V0.basis
    knodes = nodes(kbasis)[1]::Vector{Float64}
    #remember optimalpolicy also returns the argmax
    kprime = tmap(k->optimalpolicy(para,V0,k).kprime,knodes)
    #Now get policies
    return Interpoland(kbasis,kprime),V0
end;

## Total Speedup

In [17]:
kprime,V = solvebellman_parallel(para,getV0(para))
@time solvebellman_parallel(para,getV0(para));

  1.634650 seconds (18.96 M allocations: 1.880 GiB, 11.90% gc time)


# Distributed Memory
## MPI
* Lot's of different tools to do distributed memory computations

* Most ubiquitous is MPI (Message Passign Interface)

* Julia has their own toolset 
    * But I've found it less convenient for economics problems

* We'll learn the MPI library 
    * Using the Julia cluster manager

## MPI Cluster Manager
* The following spawns 4 worker processes
    * They can communicate together using MPI
    
* Do not need to be on the same machine
    * Do not have access to the same memory

In [18]:
using MPIClusterManagers,Distributed
manager = MPIManager(np=4)

#spawns the additional processes
addprocs(manager)

LoadError: UndefVarError: MPIManager not defined

## @mpi_do
* MPIClusterManagers has a convenient macro `@mpi_do`

* It tells all the worker nodes to run the exact same set of code

In [ ]:
@mpi_do manager begin #manager represents the cluster we are working with
    a = 2
    b = 3
    println("For a^b I get $(a^b)")
end;

## The Communicator Objective
* A communicator object defines how processes communicate
    * Specifies which processes 
    * Gives each process a specific rank

* We'll work with the global communicator
    * i.e. all the processes
    * can be accessed via `MPI.COMM_WORLD`

* Some simple functions can get the size of the communicator
    * and the individual processes rank (counts from 0)

In [ ]:
@mpi_do manager begin
    using MPI
    comm=MPI.COMM_WORLD #comm defines who is communicating, comm_world is all the processes
    println("Hello world, I am $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))")
end

## Some Basic Communication
* The heart of MPI is how processes communicate
    * can be one to one, many to one, one to many, or many to many
    * we'll focus on a few which are particularly useful

* `Bcast!`
    * Root process (usually rank 0) sends data to all other processes

* `Gather`
    * Root process gather's data from all other processes 

* `Allgather`  
    * Same as gather but all processes receive the same data

## Bcast Example

In [ ]:
@mpi_do manager begin
    data = rand(1)
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) and my data is $data")
    MPI.Bcast!(data,0,comm) #Bcast data using root 0 and COMM_WORLD communicator
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) and my data is now  $data")
end

## Gather Example

In [ ]:
@mpi_do manager begin
    data = rand(1)
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) and my data is $data")
    gathered_data = MPI.Gather(data,0,comm) #Bcast data using root 0 and COMM_WORLD communicator
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) here is the combined data  $gathered_data")
end

## Allgather Example

In [ ]:
@mpi_do manager begin
    data = rand(1)
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) and my data is $data")
    gathered_data = MPI.Allgather(data,comm) #Bcast data using root 0 and COMM_WORLD communicator
    println("Hi, I'm rank $(MPI.Comm_rank(comm)) here is the combined data  $gathered_data")
end

# MPI Examples

## AR(1) Example
* Go back to the problem of sampling ergodic distribution

* If we want to sample from $100,000$ points with two processes
    * Have each draw $50,000$ and the gather the data

* Very little of the code needs to change

* Just need to make sure that each process has AR(1) code on it

In [ ]:
@mpi_do manager begin
    using Parameters
end

## Implementation

In [ ]:
@mpi_do manager begin
    using Parameters
    mutable struct AR1
        μ::Float64 #Mean of the AR(1)
        ρ::Float64 #persistence of the AR(1)
        σ::Float64 #standard deviaiton of the AR(1)
    end


    """
    simulateAR1(ar,x0,T)

    Simulates an AR(1) ar for T periods with initial value x0
    """
    function simulateAR1(ar,x0,T)
        @unpack σ,μ,ρ = ar #note order doesn't matter
        x = zeros(T+1)# initialize
        x[1] = x0
        for t in 1:T
            x[t+1] = (1-ρ)*μ + ρ*x[t] + σ*randn()
        end
        return x[2:end]
    end


    #get a sample of 100_000 of x after 100 periods
    """
        draw_endofT_sample(ar,N,T)

    Draws N values of x after T periods where x follows AR1 with x[1] = 0.
    """
    function draw_endofT_sample(ar,N,T)
        sample = zeros(N)
        for i in 1:N
            sample[i] = simulateAR1(ar,0.,T)[end]
        end
        return sample
    end
end;

## Constructing the sample

In [ ]:
@mpi_do manager begin 
    ar = AR1(1.,.99,0.01)
    sample = draw_endofT_sample(ar,50000,1000) #note each process draws its own sample 
    #How to get them all together
end;

* Use Allgather (or gather) to get all the sample in one process

In [ ]:
@mpi_do manager begin
    samples = MPI.Allgather(sample,comm) #All the processes gather data
    println(sum(samples))
end

## Value Function Iteration
* Using value function iteration requires a little bit more planning

* Want to parallelize applying bellman map over state space gridpoints
    * This is a vector of length $N$

* Each process should only be computing values at subset of gridpoints 
    * Combined with others at the end

* Easiest to do then $S$ (# of processes) divides  $N$ evenly: $N=S\times K$
    * Rank 0 process takes first $K$ elements
    * Rank 1 process takes elements $K+1$ to $2K$ 
    * Rank 2 process takes elements $2K+1$ to $3K$
    * ...
    * Rank S process takes elements $(S-1)K+1$ to $SK$

## Common Code

In [ ]:
@mpi_do manager begin
    using BasisMatrices,Optim,LinearAlgebra
    @with_kw mutable struct NCParameters
        A::Float64 = 1.
        α::Float64 = 0.3
        β::Float64 = 0.96
        kgrid::Vector{Float64} = LinRange(0.05,0.5,20)
        spline_order::Int = 3
    end
    
    
    """
        optimalpolicy(para::NCParameters,Vprime,k)

    Computes optimal policy using continuation value function V and current capital
    level k given parameters in para.
    """
    function optimalpolicy(para::NCParameters,Vprime,k)
        @unpack A,α,β,kgrid = para
        k_bounds = [kgrid[1],kgrid[end]]
        f_objective(kprime) = -( log(A*k^α-kprime)+β*Vprime(kprime) ) #stores objective as function
        k_max = min(A*k^α-.001,k_bounds[2]) #Can't have negative consumptions
        result = optimize(f_objective,k_bounds[1],k_max)
        return (kprime = result.minimizer,V=-result.minimum) #using named tuples 
    end

    """
        getV0(para::NCParameters)

    Initializes V0(k) = 0 using the kgrid of para
    """
    function getV0(para::NCParameters)
        @unpack kgrid,spline_order = para

        kbasis = Basis(SplineParams(kgrid,0,spline_order))

        return Interpoland(kbasis,k->0 .*k)
    end
end;

## MPI Bellman Map Code

In [ ]:
@mpi_do manager begin
    """
    bellmanmap_mpi(Vprime,para::NCParameters)

    Apply the bellman map given continuation value function Vprime
    """
    function bellmanmap_mpi(para::NCParameters,Vprime::Interpoland)
        comm = MPI.COMM_WORLD
        r,s = MPI.Comm_rank(comm),MPI.Comm_size(comm) #get size of rank of mpi
        kbasis = Vprime.basis
        #sometimes it's helpful to tell julia what type a variable is
        knodes = nodes(kbasis)[1]::Vector{Float64}
        Nk = Int(length(knodes)/s) #note will spit out error if not equally divisible
        mynodes = knodes[1+r*Nk:(r+1)*Nk]#select a range of size Nk from all the nodes

        my_V = map(k->optimalpolicy(para,Vprime,k).V,mynodes)#only compute value function on my nodes

        #Gather at the end
        V = MPI.Allgather(my_V,comm)#will be a vector for each process 
        return Interpoland(kbasis,V) #each process constructs own value function
    end
end;

## MPI Solve Bellman Code

In [ ]:
@mpi_do manager begin    
    """
        solvebellman_mpi(para::NCParameters,V0::Interpoland)

    Solves the bellman equation for a given V0
    """
    function solvebellman_mpi(para::NCParameters,V0::Interpoland)
        comm = MPI.COMM_WORLD
        r,s = MPI.Comm_rank(comm),MPI.Comm_size(comm) #get size of rank of mpi
        diff = 1
        #Iterate of Bellman Map until difference in coefficients goes to zero
        while diff > 1e-6
            V = bellmanmap_mpi(para,V0)
            diff = norm(V.coefs-V0.coefs,Inf)
            V0 = V
        end
        kbasis = V0.basis
        knodes = nodes(kbasis)[1]::Vector{Float64}
        Nk = Int(length(knodes)/s) #note will spit out error if not equally divisible
        mynodes = knodes[1+r*Nk:(r+1)*Nk]#select a range of size Nk from all the nodes

        #remember optimalpolicy also returns the argmax
        my_kprime = map(k->optimalpolicy(para,V0,k).kprime,mynodes)
        kprime = MPI.Allgather(my_kprime,comm)#will be a vector for each process 
        #Now get policies
        return Interpoland(kbasis,kprime),V0
    end
end

## Running the Parallel Code

In [ ]:
@mpi_do manager begin
    #Note need to call solvebellman on all processes so they work in 
    #parallel
    para = NCParameters()
    para.kgrid = LinRange(0.05,0.5,102) #102 insures 104 nodes with cubic interpolation
    kprime,V = solvebellman_mpi(para,getV0(para))
end
@time @mpi_do manager begin
    solvebellman_mpi(para,getV0(para))
end